When you have multiple types of suffixes in PDB files, the main script to run FoldSeek casues an issue. The reasons is that experimentally validated structures of proteins can have multiple chains. These are indicated in the PDB files and parsed in the final csv by adding the suffix '_A', '_B', etc. The predicted structures only have one chain, and so a suffix with an underscore (usually produced for pdb from AlphaFold database), such as '_v4', casues issues. 

The result is not being able to make the column 'QueryProteinLength'. 

To circumvent this problem, the code below accepts the suffixes specified ('_A', '_B', '_C', '_D', '_v4'), and those that do not have suffixes, and supplement the csv file produced by the 'main.py' script to run FoldSeek with the column 'QueryProteinLength'. 

In [1]:
import pandas as pd
import pandas as pd
import subprocess
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SearchIO
import numpy
from Bio import PDB
import os
import warnings
from Bio.PDB.PDBExceptions import PDBConstructionWarning
from Bio.PDB import PDBParser
import time
import argparse

In [2]:
def calculate_protein_length_and_chains(pdb_file_path):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(os.path.basename(pdb_file_path), pdb_file_path)
    length = 0
    num_chains = 0
    for model in structure:
        for chain in model:
            num_chains += 1
            for residue in chain:
                if residue.id[0] == ' ':
                    length += 1
    return length, num_chains

In [3]:
def pdb_file_exists(queryUnique, directory):
    # List of possible suffixes
    suffixes = ['', '_A', '_B', '_C', '_D', '_v4', '_Ac-241209', '_truncated-241209', '_7-241209', '_Ac-241209']
    
    # Check for each possible filename
    for suffix in suffixes:
        expected_filename = f"{queryUnique}{suffix}.pdb"
        pdb_file_path = os.path.join(directory, expected_filename)
        
        if os.path.isfile(pdb_file_path):
            return pdb_file_path
    
    # Return None if no file is found
    return None

In [4]:
csv_file_directory = '/Volumes/behavgenom$/John/data_exp_info/BT/FoldSeek/commercial_proteins/2_FoldSeekRaw'
pdb_file_path = '/Volumes/behavgenom$/John/data_exp_info/BT/FoldSeek/commercial_proteins/1_commerical_proteins_processing/all_commercial_proteins/all_commercial_pdbs'
finished_foldseek_directory = '/Volumes/behavgenom$/John/data_exp_info/BT/FoldSeek/commercial_proteins/1_commerical_proteins_processing/all_commercial_proteins/all_commercial_pdbs'

In [5]:
# Read the CSV file
FirstFoldSeekdf = pd.read_csv(f'{csv_file_directory}/FirstFoldSeekdf.csv')

query_list = FirstFoldSeekdf['query'].unique()
print(query_list)

['4jox' 'AF3-cry9C-241206' 'AF3-cry1C-241206' 'AF3-mCry51Aa2-241210'
 '7ear' 'AF3-ecry3.1Ab-241206' 'AF3-Cry1A-241210' '4arx_D' '4arx_A'
 '4arx_C' '4arx_B' 'AF3-vip3A-241210' 'AF-C1IW73-F1-model_v4'
 'AF3-cry1A.105-241209' 'AF3-pinII-241210' 'AF3-cry1Ab-241206'
 'AF3-API-241210' 'AF3-vip3Aa19-241210' 'AF3-Cry2Ab2-241210'
 'AF3-cry1Ab_truncated-241209' 'AF3-mocry1F-241209'
 'AF-A0A1H3UHV3-F1-model_v4' '1ji6' 'AF3-mcry3A-241210'
 'AF3-cry1Da_7-241209' '4jp0' 'AF3-CpTI-241210' 'AF3-cry14Ab-1.b-241209'
 'AF-G3LT33-F1-model_v4' 'AF3-cry1Ab_Ac-241209' 'AF3-cry1F-241206'
 'AF3-ipd079Ea-241210' 'AF3-vip3Aa20-241210']


In [6]:
# Initialize columns
FirstFoldSeekdf['QueryProteinLength'] = 0
FirstFoldSeekdf['TargetProteinLength'] = FirstFoldSeekdf['tseq'].apply(lambda x: len(x))

# Loop through each unique query
for query in list(FirstFoldSeekdf['query'].unique()):
    queryUnique = query.split('_')[0]
    pdb_file_path = pdb_file_exists(queryUnique, finished_foldseek_directory)
    
    if pdb_file_path:
        total_length, num_chains = calculate_protein_length_and_chains(pdb_file_path)
        ProteinLength = total_length / num_chains if num_chains > 0 else 0
        FirstFoldSeekdf.loc[FirstFoldSeekdf['query'] == query, 'QueryProteinLength'] = ProteinLength
    else:
        print(f'PDB file not found for query: {queryUnique}')

# Save the modified DataFrame to a new CSV file
FirstFoldSeekdf.to_csv(f'{csv_file_directory}/FirstFoldSeekdf_mod.csv', index=False)

print(f'FoldSeek has finished running and the resulting csv file has been saved to {csv_file_directory}/FirstFoldSeekdf_mod.csv')

FoldSeek has finished running and the resulting csv file has been saved to /Volumes/behavgenom$/John/data_exp_info/BT/FoldSeek/commercial_proteins/2_FoldSeekRaw/FirstFoldSeekdf_mod.csv


In [8]:
unique_queries = FirstFoldSeekdf['query'].unique()

# Create a new DataFrame with unique queries
unique_queries_df = pd.DataFrame(unique_queries, columns=['query'])

# Merge with the original DataFrame to get the 'QueryProteinLength'
result_df = unique_queries_df.merge(FirstFoldSeekdf[['query', 'QueryProteinLength']], on='query', how='left').drop_duplicates()

# Display the resulting DataFrame
print(result_df)

result_df.to_csv(f'{csv_file_directory}/FoldSeek_results.csv', index=False)

                             query  QueryProteinLength
0                             4jox                 118
1822              AF3-cry9C-241206                 627
3818              AF3-cry1C-241206                 649
5994          AF3-mCry51Aa2-241210                 293
8596                          7ear                 584
10624         AF3-ecry3.1Ab-241206                 611
12865             AF3-Cry1A-241210                 117
13627                       4arx_D                 578
15688                       4arx_A                 578
17891                       4arx_C                 578
20049                       4arx_B                 578
22268             AF3-vip3A-241210                 789
25542        AF-C1IW73-F1-model_v4                 609
28031         AF3-cry1A.105-241209                 605
30607             AF3-pinII-241210                 153
30978            AF3-cry1Ab-241206                 608
33694               AF3-API-241210                 181
35497     